<a href="https://colab.research.google.com/github/josephineHonore/AIF360/blob/master/colab_examples/colab_workshop_adversarial_debiasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Setup

This section configures your environment to be able to run this notebook on Google Colab. Before you run this notebook, make sure you are running in a python 3 environment. You can change your runtime environment by choosing 
> Runtime > Change runtime type

in the menu.



In [1]:
!python --version

Python 3.6.8


In [0]:
# This notebook runs in Tensorflow 1.x. Soon will default be 2.x in Colab.
%tensorflow_version 1.x

In [3]:
!pip install -q -U \
  aif360==0.2.2 \
  tqdm==4.38.0 \
  tensorflow==1.15 \
  numpy==1.17.4 \
  matplotlib==3.1.1 \
  pandas==0.25.3 \
  scipy==1.3.2 \
  scikit-learn==0.21.3 \
  cvxpy==1.0.25 \
  scs==2.1.0 \
  numba==0.42.0 \
  networkx==2.4  \
  imgaug==0.2.6 \
  BlackBoxAuditing==0.1.54 \
  lime==0.1.1.36 \
  adversarial-robustness-toolbox==1.0.1

     |████████████████████████████████| 56.4MB 42kB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 163kB 63.9MB/s 
     |████████████████████████████████| 3.2MB 45.5MB/s 
     |████████████████████████████████| 634kB 59.5MB/s 
     |████████████████████████████████| 2.6MB 48.0MB/s 
     |████████████████████████████████| 276kB 52.9MB/s 
     |████████████████████████████████| 378kB 59.7MB/s 


## Notes
- The above pip command is created using AIF360's [requirements.txt](https://github.com/josephineHonore/AIF360/blob/master/requirements.txt). At the moment, the job to update these libraries is manual.
- The original notebook uses Markdown to display formated text. Currently this is [unsupported](https://github.com/googlecolab/colabtools/issues/322) in Colab.
- The tensorflow dependency is not needed for all other notebooks.
- We have changed TensorFlow's logging level to `ERROR`, just after the import of the library, to limit the amount of logging shown to the user.
- We have added code to fix the random seeds for reproducibility

In [0]:
def printb(text):
  """Auxiliar function to print in bold.
    Compensates for bug in Colab that doesn't show Markdown(diplay('text'))
  """
  print('\x1b[1;30m'+text+'\x1b[0m')

# Start of Original Notebook

#### This notebook demonstrates the use of adversarial debiasing algorithm to learn a fair classifier.
Adversarial debiasing [1] is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary's ability to determine the protected attribute from the predictions. This approach leads to a fair classifier as the predictions cannot carry any group discrimination information that the adversary can exploit. We will see how to use this algorithm for learning models with and without fairness constraints and apply them on the Adult dataset.

In [0]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow as tf

In [0]:
tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)
SEED = 42

#### Load dataset and set options

In [7]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True, seed=SEED)

/usr/local/lib/python3.6/dist-packages/aif360/datasets/standard_dataset.py:142: FutureWarning: outer method for ufunc <ufunc 'equal'> is not implemented on pandas objects. Returning an ndarray, but in the future this will raise a 'NotImplementedError'. Consider explicitly converting the Series to an array with '.array' first.
  df[label_name]))


In [8]:
# print out some labels, names, etc.
#display(Markdown("#### Training Dataset shape"))
printb('#### Training Dataset shape')
print(dataset_orig_train.features.shape)
#display(Markdown("#### Favorable and unfavorable labels"))
printb("#### Favorable and unfavorable labels")
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
#display(Markdown("#### Protected attribute names"))
printb("#### Protected attribute names")
print(dataset_orig_train.protected_attribute_names)
#display(Markdown("#### Privileged and unprivileged protected attribute values"))
printb("#### Privileged and unprivileged protected attribute values")
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
#display(Markdown("#### Dataset feature names"))
printb("#### Dataset feature names")
print(dataset_orig_train.feature_names)

#### Training Dataset shape
(34189, 18)
#### Favorable and unfavorable labels
1.0 0.0
#### Protected attribute names
['sex', 'race']
#### Privileged and unprivileged protected attribute values
[array([1.]), array([1.])] [array([0.]), array([0.])]
#### Dataset feature names
['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


#### Metric for original training data

In [9]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
#display(Markdown("#### Original training dataset"))
printb("#### Original training dataset")
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset
Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.195979
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.191102


In [10]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
#display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
printb("#### Scaled dataset - Verify that the scaling does not affect the group label statistics")
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics
Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.195979
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.191102


### Learn plan classifier without debiasing

In [0]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
tf.set_random_seed(SEED)
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess,
                          seed=SEED)

In [12]:
plain_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.737970
epoch 0; iter: 200; batch classifier loss: 0.438434
epoch 1; iter: 0; batch classifier loss: 0.376381
epoch 1; iter: 200; batch classifier loss: 0.418228
epoch 2; iter: 0; batch classifier loss: 0.385003
epoch 2; iter: 200; batch classifier loss: 0.609906
epoch 3; iter: 0; batch classifier loss: 0.352963
epoch 3; iter: 200; batch classifier loss: 0.464395
epoch 4; iter: 0; batch classifier loss: 0.438074
epoch 4; iter: 200; batch classifier loss: 0.466126
epoch 5; iter: 0; batch classifier loss: 0.423859
epoch 5; iter: 200; batch classifier loss: 0.453185
epoch 6; iter: 0; batch classifier loss: 0.415533
epoch 6; iter: 200; batch classifier loss: 0.472421
epoch 7; iter: 0; batch classifier loss: 0.499586
epoch 7; iter: 200; batch classifier loss: 0.404337
epoch 8; iter: 0; batch classifier loss: 0.476573
epoch 8; iter: 200; batch classifier loss: 0.430814
epoch 9; iter: 0; batch classifier loss: 0.383572
epoch 9; iter: 200; batch classi

In [0]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [14]:
# Metrics for the dataset from plain model (without debiasing)
#display(Markdown("#### Plain model - without debiasing - dataset metrics"))
printb("#### Plain model - without debiasing - dataset metrics")
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

#display(Markdown("#### Plain model - without debiasing - classification metrics"))
printb("#### Plain model - without debiasing - classification metrics")
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics
Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.230944
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.233361
#### Plain model - without debiasing - classification metrics
Test set: Classification accuracy = 0.801679
Test set: Balanced classification accuracy = 0.666963
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.484879
Test set: Average odds difference = -0.305113
Test set: Theil_index = 0.172990


### Apply in-processing algorithm based on adversarial learning

In [0]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()
tf.set_random_seed(SEED)

In [0]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess,
                          seed=SEED)

In [17]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.737970; batch adversarial loss: 0.666912
epoch 0; iter: 200; batch classifier loss: 0.439217; batch adversarial loss: 0.650681
epoch 1; iter: 0; batch classifier loss: 0.395579; batch adversarial loss: 0.653747
epoch 1; iter: 200; batch classifier loss: 0.398832; batch adversarial loss: 0.632667
epoch 2; iter: 0; batch classifier loss: 0.393730; batch adversarial loss: 0.596025
epoch 2; iter: 200; batch classifier loss: 0.647770; batch adversarial loss: 0.622425
epoch 3; iter: 0; batch classifier loss: 0.387068; batch adversarial loss: 0.639398
epoch 3; iter: 200; batch classifier loss: 0.555255; batch adversarial loss: 0.648315
epoch 4; iter: 0; batch classifier loss: 0.496626; batch adversarial loss: 0.605289
epoch 4; iter: 200; batch classifier loss: 0.536177; batch adversarial loss: 0.627946
epoch 5; iter: 0; batch classifier loss: 0.517603; batch adversarial loss: 0.672201
epoch 5; iter: 200; batch classifier loss: 0.520227; batch adversa

In [0]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [19]:
# Metrics for the dataset from plain model (without debiasing)
#display(Markdown("#### Plain model - without debiasing - dataset metrics"))
printb("#### Plain model - without debiasing - dataset metrics")
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
#display(Markdown("#### Model - with debiasing - dataset metrics"))
printb("#### Model - with debiasing - dataset metrics")
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



#display(Markdown("#### Plain model - without debiasing - classification metrics"))
printb("#### Plain model - without debiasing - classification metrics")
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



#display(Markdown("#### Model - with debiasing - classification metrics"))
printb("#### Model - with debiasing - classification metrics")
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics
Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.230944
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.233361
#### Model - with debiasing - dataset metrics
Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.087395
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.086066
#### Plain model - without debiasing - classification metrics
Test set: Classification accuracy = 0.801679
Test set: Balanced classification accuracy = 0.666963
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.484879
Test set: Average odds difference = -0.305113
Test set: Theil_index = 0.172990
#### Model - with debiasing - classification metrics
Test set: Classification accuracy = 0.790555
Test set: Balanced classification accuracy = 0.667707
Test set: Disparate impact = 0.583651
Test s


    References:
    [1] B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning," 
    AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.

# Exploring the results

Let's take a deeper look at the previous results.

In [0]:
#@title Code to define `print_table` function to show results in tabular format
from IPython.display import HTML, display

def print_table(headers,data,caption=""):
  """
  Prints a table given headers and data

  Inputs:
    - headers: a list of N headers
    - data: a list of N-element lists containing the data to display
    - caption: a string describing the data

  Outputs:
    - A HTML display of the table

  Example:
    caption = "A caption"
    headers = ["row","title 1", "title 2"]
    data = [["first row", 1, 2], ["second row", 2, 3]]

    print_table(headers,data,caption)
    

         A Caption
    -----------------------------------
    | row         | title 1 | title 2 |
    -----------------------------------
    | first row   | 1       | 2       |
    -----------------------------------
    | second row  | 2       | 3       |
    -----------------------------------
  """
  display(HTML(
    '<table border="1"><caption>{0}</caption><tr>{1}</tr><tr>{2}</tr></table>'.format(
        caption,
        '<th>{}</th>'.format('</th><th>'.join(line for line in headers)),
        '</tr><tr>'.join(
            '<td>{}</td>'.format(
                '</td><td>'.join(
                    str(_) for _ in row)) for row in data))
  ))

In [21]:
table = [["Train set",metric_dataset_nodebiasing_train.mean_difference(),metric_dataset_debiasing_train.mean_difference()],
         ["Test set",metric_dataset_nodebiasing_test.mean_difference(),metric_dataset_debiasing_test.mean_difference()]]
headers = ['Statistical parity difference','Without debiasing','With debiasing']
caption = "Difference in mean outcomes between unprivileged and privileged groups"

print_table(headers,table,caption)

Statistical parity difference,Without debiasing,With debiasing
Train set,-0.23094425483503983,-0.08739516361488235
Test set,-0.2333605553287056,-0.08606641998730033


We observe a big reduction in the statistical parity difference by training with Adversarial learning debias mitigation. 

Let's look at the result of this technique by evaluating other fairness metrics.

In [22]:
metrics_final = [["Accuracy", "%f" % classified_metric_nodebiasing_test.accuracy(), "%f" % classified_metric_debiasing_test.accuracy()],
                ["Balanced classification accuracy","%f" % bal_acc_nodebiasing_test, "%f" % bal_acc_debiasing_test],
                ["Disparate impact","%f" % classified_metric_nodebiasing_test.disparate_impact(), "%f" % classified_metric_debiasing_test.disparate_impact()],
                ["Equal opportunity difference", "%f" % classified_metric_nodebiasing_test.equal_opportunity_difference(), "%f" % classified_metric_debiasing_test.equal_opportunity_difference()],
                ["Average odds difference", "%f" % classified_metric_nodebiasing_test.average_odds_difference(), "%f" % classified_metric_debiasing_test.average_odds_difference()],
                ["Theil_index", "%f" % classified_metric_nodebiasing_test.theil_index(), "%f" % classified_metric_debiasing_test.theil_index()]]
headers_final = ["Classification metric", "Without debiasing","With debiasing"]
caption_final = "Difference in model performance by using Adversarial Learning mitigation"

print_table(headers_final, metrics_final, caption_final)

Classification metric,Without debiasing,With debiasing
Accuracy,0.801679,0.790555
Balanced classification accuracy,0.666963,0.667707
Disparate impact,0.000000,0.583651
Equal opportunity difference,-0.484879,-0.056681
Average odds difference,-0.305113,-0.036982
Theil_index,0.172990,0.171585


It is hard to remember the definition and the ideal expected value for each metric. We can use [explainers](https://aif360.readthedocs.io/en/latest/modules/explainers.html#) to explain each metric. There are two kind of flavours: TEXT and JSON. The JSON explainers provide structured explanations that can be used to present information to the users. Here are some examples. 

In [0]:
#@title Define `format_json` function for pretty print of JSON explainers
import json
from collections import OrderedDict

def format_json(json_str):
    return json.dumps(json.loads(json_str, object_pairs_hook=OrderedDict), indent=2)

In [0]:
from aif360.explainers import MetricJSONExplainer

In [0]:
# Define explainers for the metrics with and without debiasing
ex_nondebias_test = MetricJSONExplainer(classified_metric_nodebiasing_test)
ex_debias_test = MetricJSONExplainer(classified_metric_debiasing_test)

Now let's print the explainers for the metrics we used above. Make sure you read the whole text.

In [26]:
printb("Nondebiasing")
print(format_json(ex_nondebias_test.accuracy()))

printb("Debiasing")
print(format_json(ex_debias_test.accuracy()))

Nondebiasing
{
  "metric": "Accuracy",
  "message": "Classification accuracy (ACC): 0.8016788370982052",
  "numTruePositives": 1427.0,
  "numTrueNegatives": 10320.0,
  "numPositives": 3474.0,
  "numNegatives": 11179.0,
  "description": "Computed as (true positive count + true negative count)/(positive_count + negative_count).",
  "ideal": "The ideal value of this metric is 1.0"
}
Debiasing
{
  "metric": "Accuracy",
  "message": "Classification accuracy (ACC): 0.7905548351873336",
  "numTruePositives": 1508.0,
  "numTrueNegatives": 10076.0,
  "numPositives": 3474.0,
  "numNegatives": 11179.0,
  "description": "Computed as (true positive count + true negative count)/(positive_count + negative_count).",
  "ideal": "The ideal value of this metric is 1.0"
}


In [27]:
printb("Nondebiasing")
print(format_json(ex_nondebias_test.disparate_impact()))

printb("Debiasing")
print(format_json(ex_debias_test.disparate_impact()))

Nondebiasing
{
  "metric": "Disparate Impact",
  "message": "Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.0",
  "numPositivePredictionsUnprivileged": 531.0,
  "numUnprivileged": 4857.0,
  "numPositivePredictionsPrivileged": 2943.0,
  "numPrivileged": 9796.0,
  "description": "Computed as the ratio of rate of favorable outcome for the unprivileged group to that of the privileged group.",
  "ideal": "The ideal value of this metric is 1.0 A value < 1 implies higher benefit for the privileged group and a value >1 implies a higher benefit for the unprivileged group."
}
Debiasing
{
  "metric": "Disparate Impact",
  "message": "Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.5836510369404474",
  "numPositivePredictionsUnprivileged": 531.0,
  "numUnprivileged": 4857.0,
  "numPositivePredictionsPrivileged":

In [28]:
printb("Nondebiasing")
print(format_json(ex_nondebias_test.equal_opportunity_difference()))

printb("Debiasing")
print(format_json(ex_debias_test.equal_opportunity_difference()))

Nondebiasing
{
  "metric": "True Positive Rate Difference",
  "message": "True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.48487937478763166",
  "numTruePositivesUnprivileged": 0.0,
  "numPositivesUnprivileged": 531.0,
  "numTruePositivesPrivileged": 1427.0,
  "numPositivesPrivileged": 2943.0,
  "description": "This metric is computed as the difference of true positive rates between the unprivileged and the privileged groups.  The true positive rate is the ratio of true positives to the total number of actual positives for a given group.",
  "ideal": "The ideal value is 0. A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group."
}
Debiasing
{
  "metric": "True Positive Rate Difference",
  "message": "True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.0566814676595

In [29]:
printb("Nondebiasing")
print(format_json(ex_nondebias_test.average_odds_difference()))

printb("Debiasing")
print(format_json(ex_debias_test.average_odds_difference()))

Nondebiasing
{
  "metric": "Average Odds Difference",
  "message": "Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): -0.30511296916822117",
  "numFalsePositivesUnprivileged": 0.0,
  "numNegativesUnprivileged": 4326.0,
  "numTruePositivesUnprivileged": 0.0,
  "numPositivesUnprivileged": 531.0,
  "numFalsePositivesPrivileged": 859.0,
  "numNegativesPrivileged": 6853.0,
  "numTruePositivesPrivileged": 1427.0,
  "numPositivesPrivileged": 2943.0,
  "description": "Computed as average difference of false positive rate (false positives / negatives) and true positive rate (true positives / positives) between unprivileged and privileged groups.",
  "ideal": "The ideal value of this metric is 0.  A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group."
}
Debiasing
{
  "metric": "Average Odds Difference",
  "message": "Average odds difference (average of TPR difference and FPR di

In [30]:
printb("Nondebiasing")
print(format_json(ex_nondebias_test.theil_index()))

printb("Debiasing")
print(format_json(ex_debias_test.theil_index()))

Nondebiasing
{
  "metric": "Theil Index",
  "message": "Theil index (generalized entropy index with alpha = 1): 0.1729900486204719",
  "description": "Computed as the generalized entropy of benefit for all individuals in the dataset, with alpha = 1. It measures the inequality in benefit allocation for individuals.",
  "ideal": "A value of 0 implies perfect fairness."
}
Debiasing
{
  "metric": "Theil Index",
  "message": "Theil index (generalized entropy index with alpha = 1): 0.17158484394822954",
  "description": "Computed as the generalized entropy of benefit for all individuals in the dataset, with alpha = 1. It measures the inequality in benefit allocation for individuals.",
  "ideal": "A value of 0 implies perfect fairness."
}


# Excercises and questions

Let's make sure you understand what you just did while working on this notebook.

1. Rerun this notebook with `race` as the protected attribute. How different are the results on the fairness metrics?
2. What does the `Adversarial Debiasing` technique do?
3. What kind of classifier is this technique using? What hyperparameters could you tune?
4. Can I use the current implementation to optimize for several protected attributes?
